### Test BDD

In [ ]:
import os
import psycopg2
import pandas as pd
import numpy
import sqlalchemy
from sqlalchemy import create_engine 

#### Création BDD

In [ ]:
mot_passe = os.environ.get('pg_psw')


In [ ]:
conn = psycopg2.connect(
   database="postgres", user='postgres', password=mot_passe, host='localhost', port= '5432'
)
conn.autocommit = True

cursor = conn.cursor()

sql = '''CREATE database ProjetSPQR''';

cursor.execute(sql)
print("Base de données créée avec succès !")

conn.close()


In [ ]:
conn = psycopg2.connect(
   database="projetspqr", user='postgres', password=mot_passe, host='localhost', port= '5432'
)
conn.autocommit = True

cursor = conn.cursor()

#### Création des tables

In [ ]:
def creer_table(conn, sql_creation_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_creation_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la création de la table")
        print(e)
        return
    cursor.close()
    print("La table a été crée avec succès")

In [ ]:
sql_creer_table_personne = """
    CREATE TABLE IF NOT EXISTS personne (
    id text,
    nom_fr text,
    praenomen text,
    nomen text,
    cognomen text,
    gens text,
    sexe text,
    date_naissance text,
    lieu_naissance text,
    date_mort text,
    lieu_mort text,
    id_pere text,
    id_mere text
    );
"""

In [ ]:
sql_creer_table_activite = """
    CREATE TABLE IF NOT EXISTS activite (
    personne_id text,
    activite text
    );
"""

In [ ]:
sql_creer_table_poste = """
    CREATE TABLE IF NOT EXISTS poste (
    personne_id text,
    poste text
    );
"""

In [ ]:
sql_creer_table_mariage = """
    CREATE TABLE IF NOT EXISTS mariage (
    id text,
    id_mari text,
    id_epouse text
    );
"""

In [ ]:
sql_creer_table_oeuvre = """
    CREATE TABLE IF NOT EXISTS oeuvre (
    id text,
    titre_fr text,
    titre_lat text,
    auteur_id text,
    genre text
);
"""

In [ ]:
creer_table(conn, sql_creer_table_personne)
creer_table(conn, sql_creer_table_activite)
creer_table(conn, sql_creer_table_poste)
creer_table(conn, sql_creer_table_mariage)
creer_table(conn, sql_creer_table_oeuvre)

#### Dataframes

In [ ]:
df_personne = pd.read_table("query_personne.csv", delimiter =",") 
print(df_personne.head()) 

In [ ]:
df_personne.rename(columns={"item": "id",
                            "itemLabel": "nom_fr",
                            "praenomenLabel": "praenomen",
                            "nomenLabel": "nomen",
                            "cognomenLabel": "cognomen",
                            "gensLabel": "gens",
                            "sexeLabel": "sexe",
                            "pere": "id_pere",
                            "mere": "id_mere"},
                  inplace=True)

In [ ]:
print(df_personne.head())

In [ ]:
df_personne['id'] = df_personne.id.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_personne['lieu_naissance'] = df_personne.lieu_naissance.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_personne['lieu_mort'] = df_personne.lieu_mort.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_personne['id_pere'] = df_personne.id_pere.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_personne['id_mere'] = df_personne.id_mere.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)

In [ ]:
df_personne

##### activite

In [ ]:
df_activite = pd.read_table("query_activite.csv", delimiter =",") 
print(df_activite.head()) 

In [ ]:
df_activite.rename(columns={"item": "personne_id",
                            "occupationLabel": "activite"},
                  inplace=True)

In [ ]:
df_activite['personne_id'] = df_activite.personne_id.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)

In [ ]:
print(df_activite.head()) 

##### poste

In [ ]:
df_poste = pd.read_table("query_poste.csv", delimiter =",") 
print(df_poste.head()) 

In [ ]:
df_poste.rename(columns={"item": "personne_id",
                            "positionLabel": "poste"},
                  inplace=True)

In [ ]:
df_poste['personne_id'] = df_poste.personne_id.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)

In [ ]:
print(df_poste.head()) 

##### oeuvre

In [ ]:
df_oeuvre = pd.read_table("query_oeuvre.csv", delimiter =",") 
print(df_oeuvre.head()) 

In [ ]:
df_oeuvre.rename(columns={"oeuvre": "id",
                          "oeuvreLabel": "titre_fr",
                          "titre": "titre_lat",
                          "auteur": "auteur_id",
                          "genreLabel": "genre"},
                  inplace=True)

In [ ]:
for index, row in df_oeuvre.iterrows():
    e = df_oeuvre.at[index, 'id']
    e.replace('http://www.wikidata.org/entity/Q', '')

In [ ]:
print(df_oeuvre.head()) 

In [ ]:
df_oeuvre['id'] = df_oeuvre.id.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_oeuvre['auteur_id'] = df_oeuvre.auteur_id.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)

In [ ]:
print(df_oeuvre.head())

##### mariage

In [ ]:
df_mariage = pd.read_table("query_mariage.csv", delimiter =",") 
print(df_mariage.head()) 

In [ ]:
df_mariage.insert(0, 'id', range(0, len(df_mariage)))


In [ ]:
df_mariage.rename(columns={"item": "id_mari",
                          "epouse": "id_epouse"},
                  inplace=True)

In [ ]:
print(df_mariage.head())

In [ ]:
df_mariage['id_mari'] = df_mariage.id_mari.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_mariage['id_epouse'] = df_mariage.id_epouse.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)

In [ ]:
print(df_mariage.head()) 

#### Insertion

In [ ]:
sql_insert_personne = """INSERT INTO personne (id, nom_fr, praenomen, nomen, cognomen, gens, sexe, date_naissance, lieu_naissance, date_mort, lieu_mort, id_pere, id_mere) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

In [ ]:
sql_insert_oeuvre = """INSERT INTO oeuvre (id, titre_fr, titre_lat, auteur_id, genre) VALUES (%s, %s, %s, %s, %s)"""

In [ ]:
sql_insert_poste = """INSERT INTO poste (personne_id, poste) VALUES (%s, %s)"""

In [ ]:
sql_insert_activite = """INSERT INTO activite (personne_id, activite) VALUES (%s, %s)"""

In [ ]:
sql_insert_mariage = """INSERT INTO mariage (id, id_mari, id_epouse) VALUES (%s, %s, %s)"""

In [ ]:
conn = psycopg2.connect(
   database="projetspqr", user='postgres', password=mot_passe, host='localhost', port= '5432'
)
conn.autocommit = True

cursor = conn.cursor()

for index, row in df_personne.iterrows():
    cursor.execute(sql_insert_personne, tuple(row))

In [ ]:
for index, row in df_oeuvre.iterrows():
    cursor.execute(sql_insert_oeuvre, tuple(row))

In [ ]:
for index, row in df_activite.iterrows():
    cursor.execute(sql_insert_activite, tuple(row))


In [ ]:
for index, row in df_poste.iterrows():
    cursor.execute(sql_insert_poste, tuple(row))


In [ ]:
for index, row in df_mariage.iterrows():
    cursor.execute(sql_insert_mariage, tuple(row))